# Introduction

This notebook explains how I calculate the metrics based on scraped block log queries. 

## Set up the environment and import constants

In [25]:
# append system path to import utils
import sys
sys.path.append('../')

In [26]:
import json
import pandas as pd

In [27]:
with open('../data/scraped_data_metrics/bot_names.json') as file:
    bot_names = json.load(file)
bot_names_series = pd.Series(bot_names)

In [4]:
from utils.data_processing import file_iterator
from utils.data_processing import check_ip_address

# Code for calcualting editor metrics

The code below shows how I calculate the metrics relating to blocked editors per month.

In [5]:
# Define functions for cleaning dataframe
def clean_df(df):
    df = df.drop_duplicates()
    # remove rows containing the column names
    df = df[df['title']!='title']
    # remove rows containing missing information on user/administrator names
    df = df[~df['title'].isna() & ~df['user'].isna()]
    # remove 'User:' string from the title column
    df['title'] = df['title'].str[5:]
    # remove rows where the administrator block and unblock themselves
    df = df[df['user'] != df['title']]
    return df

In [6]:
# Set up parameters for data extraction 
dir_path = '../data/block_logs'
extracted_columns = ['title', 'action', 'user']

In [7]:
# Initialize an empty list to store metrics
editor_metrics = []

# iterate over stored data
for year_month, df in file_iterator(dir_path, start_year=2004, end_year=2023, extract_cols=extracted_columns):
    if df is not None:
        # clean data
        df = clean_df(df)
        # Check the type of editor
        df['editor_is_ip'] = check_ip_address(df, column='title')
        df['editor_is_bot'] = df['title'].isin(bot_names_series)

        # Divide block and unblock queries
        blocks = df[(df['action'] == 'block') | (df['action'] == 'reblock')]
        unblocks = df[df['action'] == 'unblock']

        # Initialise a temporary dictory to store metrics
        metric={'month_year': year_month}

        # Calculate metrics for unblocked & blocked editors
        metric['n_editor_all'] = df['title'].nunique()
        metric['n_editor_ip'] = df[df['editor_is_ip']]['title'].nunique()
        metric['n_editor_bot'] = df[df['editor_is_bot']]['title'].nunique()
        metric['n_editor_user'] =  metric['n_editor_all'] - metric['n_editor_ip'] - metric['n_editor_bot']

        # Calculate metrics for blocked editors
        metric['n_editor_all_block'] = blocks['title'].nunique()
        metric['n_editor_ip_block'] = blocks[blocks['editor_is_ip']]['title'].nunique()
        metric['n_editor_bot_block'] = blocks[blocks['editor_is_bot']]['title'].nunique()
        metric['n_editor_user_block'] =  metric['n_editor_all_block'] - metric['n_editor_ip_block'] - metric['n_editor_bot_block']

        # Calculate for unblocked editors
        metric['n_editor_all_unblock'] = unblocks['title'].nunique()
        metric['n_editor_ip_unblock'] = unblocks[unblocks['editor_is_ip']]['title'].nunique()
        metric['n_editor_bot_unblock'] = unblocks[unblocks['editor_is_bot']]['title'].nunique()
        metric['n_editor_user_unblock'] =  metric['n_editor_all_unblock'] - metric['n_editor_ip_unblock'] - metric['n_editor_bot_unblock']

        # store the metric
        editor_metrics.append(metric)


File not found: ../data/block_logs/2004/2004-01.csv
File not found: ../data/block_logs/2004/2004-01.csv.gz
File not found: ../data/block_logs/2004/2004-02.csv
File not found: ../data/block_logs/2004/2004-02.csv.gz
File not found: ../data/block_logs/2004/2004-03.csv
File not found: ../data/block_logs/2004/2004-03.csv.gz
File not found: ../data/block_logs/2004/2004-04.csv
File not found: ../data/block_logs/2004/2004-04.csv.gz
File not found: ../data/block_logs/2004/2004-05.csv
File not found: ../data/block_logs/2004/2004-05.csv.gz
File not found: ../data/block_logs/2004/2004-06.csv
File not found: ../data/block_logs/2004/2004-06.csv.gz
File not found: ../data/block_logs/2004/2004-07.csv
File not found: ../data/block_logs/2004/2004-07.csv.gz
File not found: ../data/block_logs/2004/2004-08.csv
File not found: ../data/block_logs/2004/2004-08.csv.gz
File not found: ../data/block_logs/2004/2004-09.csv
File not found: ../data/block_logs/2004/2004-09.csv.gz
File not found: ../data/block_logs/20

Extracting: ../data/block_logs/2005/2005-07.csv.gz
Extracting: ../data/block_logs/2005/2005-08.csv.gz
Extracting: ../data/block_logs/2005/2005-09.csv.gz
Extracting: ../data/block_logs/2005/2005-10.csv.gz
Extracting: ../data/block_logs/2005/2005-11.csv.gz
Extracting: ../data/block_logs/2005/2005-12.csv.gz
Extracting: ../data/block_logs/2006/2006-01.csv.gz
Extracting: ../data/block_logs/2006/2006-02.csv.gz
Extracting: ../data/block_logs/2006/2006-03.csv.gz
Extracting: ../data/block_logs/2006/2006-04.csv.gz
Extracting: ../data/block_logs/2006/2006-05.csv.gz
Extracting: ../data/block_logs/2006/2006-06.csv.gz
Extracting: ../data/block_logs/2006/2006-07.csv.gz
Extracting: ../data/block_logs/2006/2006-08.csv.gz
Extracting: ../data/block_logs/2006/2006-09.csv.gz
Extracting: ../data/block_logs/2006/2006-10.csv.gz
Extracting: ../data/block_logs/2006/2006-11.csv.gz
Extracting: ../data/block_logs/2006/2006-12.csv.gz
Extracting: ../data/block_logs/2007/2007-01.csv.gz
Extracting: ../data/block_logs/

Convert extracted metrics to dataframe

In [8]:
editor_metrics_df = pd.DataFrame(editor_metrics)
editor_metrics_df.head()

,month_year,n_editor_all,n_editor_ip,n_editor_bot,n_editor_user,n_editor_all_block,n_editor_ip_block,n_editor_bot_block,n_editor_user_block,n_editor_all_unblock,n_editor_ip_unblock,n_editor_bot_unblock,n_editor_user_unblock
0,2004-12,519,287,0,232,318,256,0,62,426,224,0,202
1,2005-01,1256,826,0,430,962,759,0,203,534,247,0,287
2,2005-02,1306,919,0,387,1216,915,0,301,336,189,0,147
3,2005-03,1118,663,0,455,1032,656,0,376,217,58,0,159
4,2005-04,1393,855,0,538,1332,851,0,481,174,62,0,112


## Save metrics to a local csv

In [9]:
editor_metrics_df.to_csv('../data/scraped_data_metrics/block_monthly_editor_metrics.csv')

# Code for calcualting administrator metrics

The code below shows how I calculate the metrics relating to administrators who blocked/unblocked users per month.

In [10]:
# Define functions for cleaning dataframe
def clean_df(df):
    df = df.drop_duplicates()
    # remove rows containing the column names
    df = df[df['title']!='title']
    # remove rows containing missing information on user/administrator names
    df = df[~df['title'].isna() & ~df['user'].isna()]
    # remove 'User:' string from the title column
    df['title'] = df['title'].str[5:]
    # remove rows where the administrator block and unblock themselves
    df = df[df['user'] != df['title']]
    return df

In [11]:
# Set up parameters for data extraction 
dir_path = '../data/block_logs'
extracted_columns = ['title', 'action', 'user']

In [12]:
# Initialize an empty list to store metrics
admin_metrics = []

# iterate over stored data
for year_month, df in file_iterator(dir_path, start_year=2004, end_year=2023, extract_cols=extracted_columns):
    if df is not None:
        # clean data
        df = clean_df(df)
        # Check the type of editor
        df['admin_is_ip'] = check_ip_address(df, column='user')
        df['admin_is_bot'] = df['user'].isin(bot_names_series)

        # Divide block and unblock queries
        blocks = df[(df['action'] == 'block') | (df['action'] == 'reblock')]
        unblocks = df[df['action'] == 'unblock']

        # Initialise a temporary dictory to store metrics
        metric={'month_year': year_month}

        # Calculate metrics for all editors
        metric['n_admin_all'] = df['user'].nunique()
        metric['n_admin_ip'] = df[df['admin_is_ip']]['user'].nunique()
        metric['n_admin_bot'] = df[df['admin_is_bot']]['user'].nunique()
        metric['n_admin_user'] =  metric['n_admin_all'] - metric['n_admin_ip'] - metric['n_admin_bot']

        # Calculate metrics for editors who block
        metric['n_admin_all_block'] = blocks['user'].nunique()
        metric['n_admin_ip_block'] = blocks[blocks['admin_is_ip']]['user'].nunique()
        metric['n_admin_bot_block'] = blocks[blocks['admin_is_bot']]['user'].nunique()
        metric['n_admin_user_block'] =  metric['n_admin_all_block'] - metric['n_admin_ip_block'] - metric['n_admin_bot_block']

        # Calculate for editors who unblock
        metric['n_admin_all_unblock'] = unblocks['user'].nunique()
        metric['n_admin_ip_unblock'] = unblocks[unblocks['admin_is_ip']]['title'].nunique()
        metric['n_admin_bot_unblock'] = unblocks[unblocks['admin_is_bot']]['title'].nunique()
        metric['n_admin_user_unblock'] =  metric['n_admin_all_unblock'] - metric['n_admin_ip_unblock'] - metric['n_admin_bot_unblock']

        # store the metric
        admin_metrics.append(metric)


File not found: ../data/block_logs/2004/2004-01.csv
File not found: ../data/block_logs/2004/2004-01.csv.gz
File not found: ../data/block_logs/2004/2004-02.csv
File not found: ../data/block_logs/2004/2004-02.csv.gz
File not found: ../data/block_logs/2004/2004-03.csv
File not found: ../data/block_logs/2004/2004-03.csv.gz
File not found: ../data/block_logs/2004/2004-04.csv
File not found: ../data/block_logs/2004/2004-04.csv.gz
File not found: ../data/block_logs/2004/2004-05.csv
File not found: ../data/block_logs/2004/2004-05.csv.gz
File not found: ../data/block_logs/2004/2004-06.csv
File not found: ../data/block_logs/2004/2004-06.csv.gz
File not found: ../data/block_logs/2004/2004-07.csv
File not found: ../data/block_logs/2004/2004-07.csv.gz
File not found: ../data/block_logs/2004/2004-08.csv
File not found: ../data/block_logs/2004/2004-08.csv.gz
File not found: ../data/block_logs/2004/2004-09.csv
File not found: ../data/block_logs/2004/2004-09.csv.gz
File not found: ../data/block_logs/20

Convert extracted metrics to dataframe

In [13]:
admin_metrics_df = pd.DataFrame(admin_metrics)
admin_metrics_df.head()

,month_year,n_admin_all,n_admin_ip,n_admin_bot,n_admin_user,n_admin_all_block,n_admin_ip_block,n_admin_bot_block,n_admin_user_block,n_admin_all_unblock,n_admin_ip_unblock,n_admin_bot_unblock,n_admin_user_unblock
0,2004-12,76,0,0,76,70,0,0,70,37,0,0,37
1,2005-01,123,0,0,123,114,0,0,114,61,0,0,61
2,2005-02,131,0,0,131,124,0,0,124,47,0,0,47
3,2005-03,164,0,0,164,154,0,0,154,57,0,0,57
4,2005-04,167,0,0,167,162,0,0,162,67,0,0,67


## Save metrics to a local csv

In [14]:
admin_metrics_df.to_csv('../data/scraped_data_metrics/block_monthly_administrator_metrics.csv')

# Code for calcualting the number of log queries (of different kinds)

The code below shows how I calculate the number of log queries of different kinds:
- number of block queries relating to users who are ip/bot/human users.
- number of block queries relating to administrators who are ip/bot/human users.
- number of block queries that are page-specific

In [15]:
# Define functions for cleaning dataframe
def clean_df(df):
    df = df.drop_duplicates()
    # remove rows containing the column names
    df = df[df['title']!='title']
    df = df[df['logid']!='logid']
    # remove rows containing missing information on user/administrator names
    df = df[~df['title'].isna() & ~df['user'].isna()]
    # remove 'User:' string from the title column
    df['title'] = df['title'].str[5:]
    # remove rows where the administrator block and unblock themselves
    df = df[df['user'] != df['title']]

    # convert page id to strings
    df['pageid'] = df['pageid'].astype(str)
    return df

In [16]:
# Set up parameters for data extraction 
dir_path = '../data/block_logs'
extracted_columns = ['logid', 'title', 'pageid', 'action', 'user']

In [17]:
# Define a function to automatically generate count variables
def count_queries(df, year_month):
    # Initialise a temporary dictory to store metrics
    metric={'month_year': year_month}

    # Calculate number of all logs
    metric['n_log_all'] = df['logid'].nunique()
    
    # Calculate number of logs that are associated with different types of editors
    metric['n_log_editor_ip'] = df[df['editor_is_ip']]['logid'].nunique()
    metric['n_log_editor_bot'] = df[df['editor_is_bot']]['logid'].nunique()
    metric['n_log_editor_user'] =  metric['n_log_all'] - metric['n_log_editor_ip'] - metric['n_log_editor_bot']

    # Calculate number of logs that are associated with different types of admins
    metric['n_log_admin_ip'] = df[df['admin_is_ip']]['logid'].nunique()
    metric['n_log_admin_bot'] = df[df['admin_is_bot']]['logid'].nunique()
    metric['n_log_admin_user'] =  metric['n_log_all'] - metric['n_log_admin_ip'] - metric['n_log_admin_bot']

    # The code below is disabled because it was unsure what the page id represents
    # Calculate number of page-specific logs
    # metric['n_queries_global'] = df[(df['pageid'] == '0') | (df['pageid'] == '0.0')]['logid'].nunique()
    # metric['n_queries_pagespecific'] = metric['n_queries_all'] - metric['n_queries_global']
    
    return metric


In [18]:
# Initialize an empty list to store metrics
log_metrics = []
log_metrics_block = []
log_metrics_unblock = []

# iterate over stored data
for year_month, df in file_iterator(dir_path, start_year=2004, end_year=2023, extract_cols=extracted_columns):
    if df is not None:
        # clean data
        df = clean_df(df)
        # Check the type of editor/administrator
        df['editor_is_ip'] = check_ip_address(df, column='title')
        df['editor_is_bot'] = df['title'].isin(bot_names_series)
        df['admin_is_ip'] = check_ip_address(df, column='user')
        df['admin_is_bot'] = df['user'].isin(bot_names_series)

        # Divide block and unblock queries
        blocks = df[(df['action'] == 'block') | (df['action'] == 'reblock')]
        unblocks = df[df['action'] == 'unblock']

        # store the metric
        log_metrics.append(count_queries(df=df, year_month=year_month))
        log_metrics_block.append(count_queries(df=blocks, year_month=year_month))
        log_metrics_unblock.append(count_queries(df=unblocks, year_month=year_month))


File not found: ../data/block_logs/2004/2004-01.csv
File not found: ../data/block_logs/2004/2004-01.csv.gz
File not found: ../data/block_logs/2004/2004-02.csv
File not found: ../data/block_logs/2004/2004-02.csv.gz
File not found: ../data/block_logs/2004/2004-03.csv
File not found: ../data/block_logs/2004/2004-03.csv.gz
File not found: ../data/block_logs/2004/2004-04.csv
File not found: ../data/block_logs/2004/2004-04.csv.gz
File not found: ../data/block_logs/2004/2004-05.csv
File not found: ../data/block_logs/2004/2004-05.csv.gz
File not found: ../data/block_logs/2004/2004-06.csv
File not found: ../data/block_logs/2004/2004-06.csv.gz
File not found: ../data/block_logs/2004/2004-07.csv
File not found: ../data/block_logs/2004/2004-07.csv.gz
File not found: ../data/block_logs/2004/2004-08.csv
File not found: ../data/block_logs/2004/2004-08.csv.gz
File not found: ../data/block_logs/2004/2004-09.csv
File not found: ../data/block_logs/2004/2004-09.csv.gz
File not found: ../data/block_logs/20

/Users/haochen/Documents/France/Projects/wikipedia/wikipedia_scrapping/notebooks/../utils/data_processing.py:58: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f_path_gz, usecols=extract_cols, compression='gzip', on_bad_lines='warn')


Extracting: ../data/block_logs/2023/2023-02.csv.gz
Extracting: ../data/block_logs/2023/2023-03.csv.gz
Extracting: ../data/block_logs/2023/2023-04.csv.gz
Extracting: ../data/block_logs/2023/2023-05.csv.gz
Extracting: ../data/block_logs/2023/2023-06.csv.gz
Extracting: ../data/block_logs/2023/2023-07.csv.gz
Extracting: ../data/block_logs/2023/2023-08.csv.gz
Extracting: ../data/block_logs/2023/2023-09.csv.gz
Extracting: ../data/block_logs/2023/2023-10.csv.gz
Extracting: ../data/block_logs/2023/2023-11.csv.gz
Extracting: ../data/block_logs/2023/2023-12.csv.gz


Convert extracted metrics to dataframe

In [19]:
log_metrics_df = pd.DataFrame(log_metrics)
log_metrics_block_df = pd.DataFrame(log_metrics_block)
log_metrics_unblock_df = pd.DataFrame(log_metrics_unblock)

In [20]:
# add suffix to the variable names end
log_metrics_block_df = log_metrics_block_df.add_suffix('_block')
log_metrics_unblock_df = log_metrics_unblock_df.add_suffix('_unblock')

In [21]:
# merge dataframes
log_metrics_all_df = pd.merge(log_metrics_df, log_metrics_block_df, left_on='month_year', right_on='month_year_block')
log_metrics_all_df = pd.merge(log_metrics_all_df, log_metrics_unblock_df, left_on='month_year', right_on='month_year_unblock')

In [22]:
# drop repeated variables
log_metrics_all_df = log_metrics_all_df.drop(['month_year_block', 'month_year_unblock'], axis=1)

## Save metrics to a local csv

In [23]:
log_metrics_all_df.to_csv('../data/scraped_data_metrics/block_monthly_log_metrics.csv')